# Demo Detection: Обработка данных, обучение, демонстрация и экспорт модели

Обучение модели было выполнено на M4 Pro MacBook Pro (24G RAM). Экспорт для
Debian c RTX5060 (8G VRAM) на борту.


In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns
from ultralytics import YOLO

from preprocessing import TestTaskDatasetPreprocessor


ROOT = "../../"


---


## Обработка данных

Исходные данные были представлены в виде директорий с кадрами из
соответствующих чанков видео. Все кадры оказались упорядочены, поэтому
построить соответствие между изображениями и аннотациями было нетрудно.
Описанный ниже класс даёт абстракцию для перевода из исходного формата в
подходящий для обучения YOLO26 от Ultralitycs.

Конкретно с нашими путями его следует использовть следущим образом. Чтобы 
упростить задачу детекции для модели, не включаем в выборку класс трейлера.


In [ ]:
prefix = ROOT + "dataset/test_task_dl_26_12_2025/train/"
frames = [
    "2025-10-03_05_vt53_criterion_1_6_day_bw_panorama_1882x862",
    "2025-10-03_05_vt53_criterion_1_8_day_bw_panorama_1882x862",
    "2025-10-05T11-12_vt53_criterion_3_1_day_bw_panorama_1882x862",
]
annotations = [
    "annotations_task_2299_2025-10-03_05_vt53_criterion_1_6_day_bw"
    + "_panorama_1882x862.xml",
    "annotations_task_2303_2025-10-03_05_vt53_criterion_1_8_day_bw"
    + "_panorama_1882x862.xml",
    "annotations_task_2310_2025-10-05T11-12_vt53_criterion_3_1_day_bw"
    + "_panorama_1882x862.xml",
]

paths_aligned = []
for dir, file in zip(frames, annotations):
    paths = (prefix + dir, prefix + file)
    paths_aligned.append(paths)

preprocessor = TestTaskDatasetPreprocessor(
    root=ROOT + "dataset/training_data_sample",
    class_map={"car": 0, "truck": 1},
    image_dims=(1882.0, 862.0),
)

preprocessor.run(paths_aligned)


---


## Обучение модели

Обучаем модель 20 эпох (+-4 часа) с наиболее передовым оптимизатором. Размер
пачки устанавливаем равным четырём (+-9.8G Unified Memory). По окончании
процесса обучения файл модели будет находиться в пути
ROOT/runs/detect/demo_detection/demo_detection_v4/best.pt. Однако промежуточные
результаты храянтся в файле проекта (ROOT/runs/detect/demo_detection) с
названием "demo_detection_v4 + номер эпохи".

--------------------------------------------------------------------------------

In [ ]:
model = YOLO(ROOT + "models/yolo26m.pt")

model.train(
    data="training_data.yaml",
    epochs=20,
    imgsz=1280,
    batch=4,
    device="mps",
    amp=True,
    cache=False,
    workers=0,
    rect=True,
    optimizer="MuSGD",
    project="demo_detection",
    name="demo_detection_v4",
)


---


## Детекция

Прогон тестового видео через модель детекции осуществляется выполнением
следующей строки в терминале:
```
python track.py
```

Подразумевается, что Вы уже находитесь в директории ROOT/src/python и уже
активировали виртуальное окружение. Если это не так, скрипт может не найти путь
к нужным файлам.

---


## Экспорт модели

Экпорт модели осуществляется в формате ONNX, чтобы обеспечить наибольшую
совместимсоть со всеми видокартами NVIDIA GT1000+ серий и облегчить процесс
реацлизации приложения на C++.


In [ ]:
model = YOLO(ROOT + "models/demo_detection_v4e13.pt")

model.export(format="onnx")

Проверим, что экспортированная модель запускается корректно.


In [ ]:
onnx_model = YOLO(ROOT + "models/demo_detection_v4e13.onnx")

results = onnx_model(
    ROOT + "dataset/training_data/images/train/"
    + "2025-10-03_05_vt53_criterion_1_6_day_bw_panorama_1882x862"
    + "_video_20251004T061524_to_20251004T061535_11.0s_frame183.jpg"
)

---


## Визуализация резульатов обучения и выводы

С самого начала для решения задачи детекции была обучена YOLO26l, однако она
не дала необходимого резульатат, так как, судя по всему, оказалась чересчур
большой, а потому училась довольно медленно и 18-и эпох было недостаточно.

Позже было решено заменить её на YOLO26m, потому что она более портативна, с
большей вероятностью даст треубемую прозиводительность на GPU с архитектурой
Pascal (GT1000 серии) и быстрее обучится, потребляя при этом меньшее количество
энергии.

На графике ниже можно увидеть сопоставление процессов обучения двух моделей.
Любопытно, что метрики, на самом деле, не так далеки друг от друга, но YOLO26m
в итоге лучше справилась с детекцией грузовика, не спутав его при этом с ТС
легкового типа.

Видео и статистики находятся в директории ROOT/models/training_results.


In [ ]:
def process_logs(file_path, model_label):
    df = pl.read_csv(file_path)

    return df.with_columns(
        [
            pl.lit(model_label).alias("model"),
            (
                (pl.col("time").diff().fill_null(pl.col("time").first())) / 60
            ).alias("epoch_time"),
        ]
    )


df_l = process_logs(
    ROOT + "models/training_results/demo_detection_v3e18.csv", "YOLO26l"
)
df_m = process_logs(
    ROOT + "models/training_results/demo_detection_v4e13.csv", "YOLO26m"
)

df_combined = pl.concat([df_l, df_m])

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(
    "Сравнение моделей: YOLO26l vs YOLO26m\n(Детекция легковых и грузовых автомобилей)",
    fontsize=20,
)

sns.lineplot(
    data=df_combined,
    x="epoch",
    y="metrics/mAP50(B)",
    hue="model",
    ax=axes[0, 0],
    marker="o",
    linewidth=2.5,
)
sns.lineplot(
    data=df_combined,
    x="epoch",
    y="metrics/mAP50-95(B)",
    hue="model",
    ax=axes[0, 0],
    marker="s",
    linestyle="--",
)
axes[0, 0].set_title(
    "Точность детекции: mAP@.5 (сплошная) и mAP@.5:.95 (пунктир)", fontsize=14
)
axes[0, 0].set_ylabel("Значение mAP")
axes[0, 0].set_xlabel("Эпоха")

sns.lineplot(
    data=df_combined,
    x="epoch",
    y="val/box_loss",
    hue="model",
    ax=axes[0, 1],
    marker="o",
    linewidth=2,
)
axes[0, 1].set_title(
    "Стабильность регрессии рамок (Box Loss на валидации)", fontsize=14
)
axes[0, 1].set_ylabel("Функция потерь (Loss)")
axes[0, 1].set_xlabel("Эпоха")

sns.lineplot(
    data=df_combined,
    x="epoch",
    y="metrics/precision(B)",
    hue="model",
    ax=axes[1, 0],
    marker="o",
)
sns.lineplot(
    data=df_combined,
    x="epoch",
    y="metrics/recall(B)",
    hue="model",
    ax=axes[1, 0],
    marker="x",
    linestyle=":",
)
axes[1, 0].set_title("Точность (сплошная) vs. Полнота (пунктир)", fontsize=14)
axes[1, 0].set_ylabel("Метрика (0-1)")
axes[1, 0].set_xlabel("Эпоха")

sns.barplot(
    data=df_combined, x="epoch", y="epoch_time", hue="model", ax=axes[1, 1]
)
axes[1, 1].set_title("Затраты времени на эпоху (в минутах)", fontsize=14)
axes[1, 1].set_ylabel("Длительность (мин)")
axes[1, 1].set_xlabel("Эпоха")

plt.tight_layout(rect=[0, 0.03, 1, 0.93])
plt.savefig(
    ROOT + "models/training_results/demo_detection_comparison_dashboard.png",
    dpi=300,
)
plt.show()
